In [1]:
# Cell 1: Setup and Install
!pip install -q mediapipe opencv-python tqdm

import warnings
warnings.filterwarnings('ignore')

import os
import cv2
import numpy as np
import mediapipe as mp
import urllib.request
from pathlib import Path
from tqdm import tqdm
from collections import defaultdict

print("✅ Setup complete")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 10.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.1.2 requires gymnasium>=1.0.0, but you have gymnasium 0.29.0 which is incompatible.


2026-02-01 08:24:21.034298: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769934261.236118      24 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769934261.295160      24 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769934261.773428      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769934261.773480      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769934261.773483      24 computation_placer.cc:177] computation placer alr

✅ Setup complete


In [2]:
# Cell 2: Download MediaPipe Models
MEDIAPIPE_DIR = Path('/kaggle/working/mediapipe_models')
MEDIAPIPE_DIR.mkdir(exist_ok=True)

URLS = {
    'pose_landmarker.task': 'https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_heavy/float16/1/pose_landmarker_heavy.task',
    'hand_landmarker.task': 'https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/1/hand_landmarker.task',
    'face_landmarker.task': 'https://storage.googleapis.com/mediapipe-models/face_landmarker/face_landmarker/float16/1/face_landmarker.task'
}

for name, url in URLS.items():
    filepath = MEDIAPIPE_DIR / name
    if not filepath.exists():
        print(f"Downloading {name}...")
        urllib.request.urlretrieve(url, filepath)

print("✅ MediaPipe models ready")

✅ MediaPipe models ready


In [3]:
# Cell 3: Define 123 Selected Classes
SELECTED_CLASSES = [
    # Greetings (9)
    'hello', 'thank you', 'good morning', 'good afternoon', 'good evening', 
    'good night', 'alright', 'how are you', 'pleased',
    
    # Pronouns (8)
    'i', 'you', 'he', 'she', 'we', 'they', 'it', 'you (plural)',
    
    # People (20)
    'mother', 'father', 'brother', 'sister', 'family', 'friend',
    'son', 'daughter', 'child', 'baby', 'boy', 'girl',
    'man', 'woman', 'adult', 'husband', 'wife',
    'grandmother', 'grandfather', 'parent',
    
    # Places (19)
    'school', 'house', 'hospital', 'market', 'bank', 'temple',
    'park', 'office', 'city', 'india', 'university', 'library',
    'restaurant', 'store or shop', 'train station', 'street or road',
    'court', 'location', 'ground',
    
    # Animals (8)
    'dog', 'cat', 'bird', 'fish', 'cow', 'horse', 'mouse', 'animal',
    
    # Colours (11)
    'red', 'blue', 'green', 'yellow', 'orange', 'pink', 'brown',
    'black', 'white', 'grey', 'colour',
    
    # Clothes (10)
    'shirt', 't-shirt', 'pant', 'dress', 'skirt', 'suit', 'hat',
    'shoes', 'pocket', 'clothing',
    
    # Transport (9)
    'car', 'bus', 'train', 'plane', 'bicycle', 'boat', 'truck',
    'transportation', 'train ticket',
    
    # Time (4)
    'time', 'year', 'night', 'second',
    
    # Adjectives (25)
    'good', 'bad', 'big large', 'small little', 'hot', 'cold', 'warm', 'cool',
    'new', 'old', 'young', 'happy', 'fast', 'slow',
    'tall', 'short', 'long', 'wide', 'narrow',
    'wet', 'dry', 'sick', 'healthy', 'quiet', 'loud'
]

print(f"✅ {len(SELECTED_CLASSES)} classes selected")

✅ 123 classes selected


In [4]:
# Cell 4: Scan INCLUDE for Selected Classes
INCLUDE_DIR = Path('/kaggle/input/include')

selected_videos = defaultdict(list)

for root, dirs, files in os.walk(INCLUDE_DIR):
    video_files = [f for f in files if f.lower().endswith(('.mov', '.mp4'))]
    if not video_files:
        continue
    
    parts = Path(root).relative_to(INCLUDE_DIR).parts
    if len(parts) >= 3:
        class_folder = parts[2]
        if '. ' in class_folder:
            class_name = class_folder.split('. ', 1)[1].strip().lower()
        else:
            class_name = class_folder.strip().lower()
        
        if class_name in SELECTED_CLASSES:
            for video in video_files:
                selected_videos[class_name].append(Path(root) / video)

print(f"✅ Found videos:")
print(f"   Classes: {len(selected_videos)}")
print(f"   Total videos: {sum(len(v) for v in selected_videos.values())}")

✅ Found videos:
   Classes: 123
   Total videos: 2508


In [5]:
# Cell 5: MediaPipe Extraction Functions
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

def create_landmarkers():
    pose = vision.PoseLandmarker.create_from_options(
        vision.PoseLandmarkerOptions(
            base_options=python.BaseOptions(model_asset_path=str(MEDIAPIPE_DIR / 'pose_landmarker.task')),
            running_mode=vision.RunningMode.IMAGE
        )
    )
    
    hand = vision.HandLandmarker.create_from_options(
        vision.HandLandmarkerOptions(
            base_options=python.BaseOptions(model_asset_path=str(MEDIAPIPE_DIR / 'hand_landmarker.task')),
            running_mode=vision.RunningMode.IMAGE,
            num_hands=2
        )
    )
    
    face = vision.FaceLandmarker.create_from_options(
        vision.FaceLandmarkerOptions(
            base_options=python.BaseOptions(model_asset_path=str(MEDIAPIPE_DIR / 'face_landmarker.task')),
            running_mode=vision.RunningMode.IMAGE
        )
    )
    
    return pose, hand, face

def extract_landmarks(video_path, pose_lm, hand_lm, face_lm):
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        return None
    
    landmarks_seq = []
    
    while cap.isOpened() and len(landmarks_seq) < 300:
        ret, frame = cap.read()
        if not ret:
            break
        
        try:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame_rgb)
            
            pose_result = pose_lm.detect(mp_image)
            hand_result = hand_lm.detect(mp_image)
            face_result = face_lm.detect(mp_image)
            
            landmarks = []
            
            # Pose: 33 × 4 = 132
            if pose_result.pose_landmarks:
                for lm in pose_result.pose_landmarks[0]:
                    landmarks.extend([lm.x, lm.y, lm.z, lm.visibility])
            else:
                landmarks.extend([0.0] * 132)
            
            # Left hand: 21 × 3 = 63
            if hand_result.hand_landmarks and len(hand_result.hand_landmarks) > 0:
                for lm in hand_result.hand_landmarks[0]:
                    landmarks.extend([lm.x, lm.y, lm.z])
            else:
                landmarks.extend([0.0] * 63)
            
            # Right hand: 21 × 3 = 63
            if hand_result.hand_landmarks and len(hand_result.hand_landmarks) > 1:
                for lm in hand_result.hand_landmarks[1]:
                    landmarks.extend([lm.x, lm.y, lm.z])
            else:
                landmarks.extend([0.0] * 63)
            
            # Face: 50 × 3 = 150
            if face_result.face_landmarks:
                key_idx = list(range(0, 468, 9))[:50]
                for idx in key_idx:
                    if idx < len(face_result.face_landmarks[0]):
                        lm = face_result.face_landmarks[0][idx]
                        landmarks.extend([lm.x, lm.y, lm.z])
                    else:
                        landmarks.extend([0.0, 0.0, 0.0])
            else:
                landmarks.extend([0.0] * 150)
            
            landmarks_seq.append(landmarks)
        except:
            continue
    
    cap.release()
    
    if len(landmarks_seq) < 10:
        return None
    
    return np.array(landmarks_seq, dtype=np.float32)

print("✅ Extraction functions ready")

✅ Extraction functions ready


In [6]:
# Cell 6: Extract All Videos (2-3 hours)
CACHE_DIR = Path('/kaggle/working/isl_cache_123')
CACHE_DIR.mkdir(exist_ok=True)

pose_lm, hand_lm, face_lm = create_landmarkers()

success = 0
failed = 0

print("="*60)
print("🔄 STARTING EXTRACTION")
print(f"   Total videos: {sum(len(v) for v in selected_videos.values())}")
print(f"   Estimated time: 2-3 hours")
print("="*60)

for class_name, videos in selected_videos.items():
    print(f"\n📂 {class_name}")
    
    for video_path in tqdm(videos, desc=f"  {class_name}"):
        cache_path = CACHE_DIR / f"{video_path.stem}.npy"
        
        if cache_path.exists():
            success += 1
            continue
        
        landmarks = extract_landmarks(video_path, pose_lm, hand_lm, face_lm)
        
        if landmarks is not None:
            np.save(cache_path, landmarks)
            success += 1
        else:
            failed += 1

pose_lm.close()
hand_lm.close()
face_lm.close()

print("\n" + "="*60)
print("✅ EXTRACTION COMPLETE")
print(f"   Success: {success}")
print(f"   Failed: {failed}")
print(f"   Cache files: {len(list(CACHE_DIR.glob('*.npy')))}")
print("="*60)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1769934278.681334      79 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1769934278.800393      79 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1769934278.841883      84 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1769934278.864303      84 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1769934278.865611      87 face_landmarker_graph.cc:174] Sets FaceBlendshapesGraph acceleration to xnnpack by default.
W0000 00:00:1769934278.874978      90 inference_feedback_manager.cc:114] Feedback mana

🔄 STARTING EXTRACTION
   Total videos: 2508
   Estimated time: 2-3 hours

📂 hot


  hot: 100%|██████████| 21/21 [02:53<00:00,  8.24s/it]



📂 warm


  warm: 100%|██████████| 21/21 [03:01<00:00,  8.63s/it]


...


📂 blue


  blue: 100%|██████████| 20/20 [02:08<00:00,  6.41s/it]



📂 green


  green: 100%|██████████| 20/20 [02:17<00:00,  6.87s/it]


✅ EXTRACTION COMPLETE
   Success: 2508
   Failed: 0
   Cache files: 2235


In [7]:
# Cell 7: Create Label Mapping
import json

label_to_id = {cls: idx for idx, cls in enumerate(sorted(selected_videos.keys()))}
id_to_label = {idx: cls for cls, idx in label_to_id.items()}

mapping = {
    'num_classes': len(label_to_id),
    'label_to_id': label_to_id,
    'id_to_label': id_to_label
}

with open('/kaggle/working/label_mapping_123.json', 'w') as f:
    json.dump(mapping, f, indent=2)

print(f"✅ Saved mapping for {len(label_to_id)} classes")
print(f"   Cache: {len(list(CACHE_DIR.glob('*.npy')))} files")
print(f"\n📤 Save /kaggle/working as Kaggle dataset!")

✅ Saved mapping for 123 classes
   Cache: 2235 files

📤 Save /kaggle/working as Kaggle dataset!
